# Create Network from Overture

Investigation into using Overture data to create a transportation network.

## Download Data 
Uses [Overture Maps Python Command Line Tool](https://docs.overturemaps.org/getting-data/overturemaps-py/)

In [ ]:
! pip install overturemaps
! pip install libgdal-arrow-parquet
! pip install ipyleaflet


### Find bounding box

In [ ]:
from ipyleaflet import Map, DrawControl, TileLayer
m = Map(center=(37.7749, -122.4194), zoom=14, layers=[TileLayer(url='https://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}{r}.png')]) # Example: San Francisco
draw_control = DrawControl()

def handle_draw(self, action, geo_json):
    if action == 'created':
        # Get the coordinates of the bounding box
        coordinates = geo_json['geometry']['coordinates'][0]
        lons = [coord[0] for coord in coordinates]
        lats = [coord[1] for coord in coordinates]
        formatted_bbox = f"{min(lons)},{min(lats)},{max(lons)},{max(lats)}"
        print("--bbox:", formatted_bbox)

draw_control.on_draw(handle_draw)

m.add_control(draw_control)
m

### Get the data using the CLI and store on local file path

In [5]:
from pathlib import Path
from typing import Literal
dl_path = Path.cwd()/'downloads'
dl_path.mkdir(exist_ok=True,parents=True)
bbox = "-122.449322,37.762098,-122.434216,37.772682"
fmt: Literal["geojson", "geojsonseq", "geoparquet"] = "geoparquet"

In [6]:
dl_segment = f"overturemaps download --bbox={bbox} -f {fmt} --type=segment -o {dl_path/'segments.parquet'}"
!{dl_segment}

In [7]:
dl_connector = f"overturemaps download --bbox={bbox} -f {fmt} --type=connector -o {dl_path/'connectors.parquet'}"
!{dl_connector}

## Review Downloaded Data

In [ ]:
from geopandas import read_file
segments_gpd = read_file(dl_path/'segments.parquet')
nodes_gpd = read_file(dl_path/'connectors.parquet')

In [ ]:
mymap = segments_gpd.explore(
    tiles='CartoDB dark_matter',
    zoom_start=16,
    column="class",
    location=segments_gpd.geometry.union_all().centroid.coords[0][::-1],
    cmap='viridis')
nodes_gpd.explore(m=mymap, color='grey')

### Available Fields

Key missing field in all data we downloaded here was `lanes` which not only determines the number of lanes but also the link directionality.

WARNING:  This data is **not suitable for use** yet because of this shortcoming.

In [ ]:
cols = ['id', 'subtype', 'names.primary', 'routes', 'class', 'geometry']
segments_gpd.loc[segments_gpd["class"]=="primary",cols ].head()

### Field Conversion

Many of the fields are nested in stringified json (necessary in order to serialize as parquet).  This requires compute-intensive computation to grab values that you might need - although does provide a great deal of flexibility in how values are scoped.

Scoped values include:
- Speed limits: which seems to just be a lookup by facility class anyway (see plot below)

In [11]:
import pandas as pd
import json
# Convert stringified JSON to Python objects
segments_gpd['speed_limits'] = segments_gpd['speed_limits'].apply(
    lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else x
)
# speed limits is an array, but I'm just taking the first value
segments_gpd['max_speed'] = segments_gpd['speed_limits'].apply(
    lambda x: x[0]['max_speed']['value'] if isinstance(x, list) and isinstance(x[0],dict) and x[0]['max_speed'] else None
)

In [ ]:
segments_gpd.dropna(subset=['max_speed', 'class']).plot(kind='scatter', x='max_speed', y='class')
#not a lot of variety - looks like they were set by algorithm

## Convert to Wrangler Format

Despite its shortcomings, we attempt to do what we can to re-format this data into network-wrangler's network format including:
1. renaming columns that are directly comparible
2. parsing columns into wrangler values (i.e. inferring "drive_access" from roadway class)
3. reviewing topological inconsistencies (e.g. duplicate AB values)
4. creating a duplicatable method to translate GERS_IDS (hex) --> wrangler ids (integers).  

NOTE: This process is left incomplete, as the inability to determine which roadways should be two-way vs one-way is impossible without the `lanes` attribute.

In [21]:
rename_cols = {
    "id":"gers_id",
    "names.primary":"name",
    "class": "roadway",
    "max_speed": "speed_limit",
    "geometry": "geometry"
}
links_df = segments_gpd.loc[:, list(rename_cols.keys())].rename(columns=rename_cols)
links_df['waypoints'] = segments_gpd['connectors'].apply(
    lambda x: json.loads(x) if pd.notna(x) and isinstance(x, str) else x
)

In [ ]:
segments_gpd[['subtype','class']].drop_duplicates()

In [23]:
links_df["rail_only"] = segments_gpd.subtype.str.contains("rail")
links_df["walk_access"] = segments_gpd['class'].isin(['footway','path','steps'])
links_df["bike_access"] = segments_gpd['class'].isin(['path','cycleway','service','tertiary','residential'])
links_df["drive_access"] = ~segments_gpd['class'].isin(['footway','path','steps','cycleway',None])
links_df["truck_access"] = links_df["drive_access"]


### Create IDS

Process to convert IDS from hex to ints as required by wrangler.

Note that we are forced to limit the number of hex digits converted to integers in order for the resulting integer to be able to be serialized in parquet which has an upper-bounds int limit based on c-int-large - but we do check that the values (at least within this dataset) are unique.

In [ ]:
# Convert IDS from hex to ints as required by wrangler
id_digits = 6 # a couple digits ...can't go BACK to gers id but should be relatively unique
hex_to_int = lambda x: int(x[-id_digits:], 16)
links_df[['A_h','B_h']] = links_df['waypoints'].apply(lambda x: pd.Series([x[0]['connector_id'], x[-1]['connector_id']]))
links_df['A'] = links_df['A_h'].apply(hex_to_int)
links_df['B'] = links_df['B_h'].apply(hex_to_int)
links_df['model_link_id'] = links_df['gers_id'].apply(hex_to_int)
assert links_df.model_link_id.nunique() == len(links_df)
links_df[['model_link_id','A','B']].head()

In [ ]:
rename_cols = {
    "id":"gers_id",
    "geometry": "geometry"
}
nodes_df = nodes_gpd.loc[:, list(rename_cols.keys())].rename(columns=rename_cols)
nodes_df['model_node_id'] = nodes_df['gers_id'].apply(hex_to_int)
# find duplicated model_node_ids
# nodes_df.loc[nodes_df.model_node_id.duplicated(keep=False), ['gers_id','model_node_id']]
assert nodes_df.model_node_id.nunique() == len(nodes_df)
nodes_df.head()


#### Check uniqueness

Find records with duplicated A-B, note that they are all footpaths and then keep the shortest of them since that is the path that would be taken anyway.

In [ ]:
# find links that have duplicated A and B values
links_df.loc[links_df.duplicated(subset=['A','B'], keep=False)]


In [59]:
# these are all sidewalks/stairs, so let's just take the one with the shortest length
links_df = links_df.sort_values('geometry').drop_duplicates(subset=['A','B'], keep='first')


#### Check topology

- find links where AB doesn't have a corresponding BA

Turns out that is basically all of them, thus we really do need "lanes" in order to know which of these to turn into two-way.


In [ ]:
# f
ba_df = links_df.rename(columns={'A': 'B', 'B': 'A'})
merged_df = pd.merge(links_df, ba_df, on=['A', 'B'], how='left', indicator=True)
ab_without_ba = merged_df[merged_df['_merge'] == 'left_only']
f"Links without corresponding link the other direction: {len(ab_without_ba)}/{len(links_df)}"

In [ ]:
ab_with_ba = merged_df[merged_df['_merge'] == 'both']
ab_with_ba.head()

## Save and Load into Wrangler

Note that this step will not work yet as it
- fails validation for requiring lanes
- doesn't have correct reversed links

In [54]:
nodes_df.to_parquet(dl_path/'nodes.parquet')
links_df.to_parquet(dl_path/'links.parquet')


In [ ]:
from network_wrangler import load_roadway_from_dir
#if we try and load right away, there are clearly some issues. So we need to clean up the data more.
road_net = load_roadway_from_dir(dl_path,file_format="parquet")